In [ ]:
%matplotlib inline  

import numpy as np
import healpy as hp
import matplotlib.pyplot as plt

# Import the NaMaster python wrapper
from astropy.io import fits
from astropy.table import Table
import treecorr
import os


import math
from scipy.optimize import curve_fit

from scipy import optimize
import scipy

from os import listdir
from os.path import isfile, join

from camb import correlations

import pickle



In [ ]:
def Cl2maps(Cl, nside, ell_factor = False):
    '''
    Generate Gaussian Random Field for one Cl. 
    Cl: 1-d array, for ell value from 1-3*N
    nside: nside of the generated map
    ell_factor: if your Cl has ell(ell+1)/2pi in front, need to set this to True
    '''
    ell_int = np.arange(1,len(Cl)+1,1)
    
    if ell_factor:
        Cl = np.divide(np.divide(Cl,ell_int),(ell_int+1))*2 * np.pi
        
    alms = hp.sphtfunc.synalm(Cl)
    maps = hp.sphtfunc.alm2map(alms, nside, pol = False)
    
    return maps
    
    
    
    

In [ ]:

def powlaw(x, a, b) :
    '''
    Power law fitting funtion, b needs to be non-negative, otherwise blows up at large theta
    x: theta, in [degree]
    '''
    return a * np.power(x, -b)

def powtan(x, a, b):
    '''
    Power law * tangential function cutoff at a constant theta. 
    x: theta, in [degree]
    '''
    
    cutoff = 1.7
    
    return a* np.power(x, -b) * ((-np.arctan(x - cutoff ))/np.pi + 1/2 )

def twoexp(x, amp1, scale1, amp2, scale2):
    '''
    amp1*exp(-scale1*x)+amp2*exp(-scale2*x)
    x: theta, in [degree]
    '''
    return amp1*np.exp(-scale1*x)+amp2*np.exp(-scale2*x)

def oneexp(x, amp, scale):
    '''
    Exponential law
    x: theta, in [degree]
    '''
    return amp*np.exp(-scale*x)

def curve_fit_log(xdata, ydata, method) :
    
    #print xdata_log, ydata_log
    if method == "powlaw":
        popt, pcov = curve_fit(powlaw, xdata, ydata , maxfev = 10000,
                              bounds = ((-np.inf, 0.0), (np.inf, np.inf)))
    elif method == "powtan":
        popt, pcov = curve_fit(powtan, xdata, ydata , maxfev = 10000,
                              bounds = ((-np.inf, 0.0), (np.inf, np.inf)))
    elif method == "twoexp":
        popt, pcov = curve_fit(twoexp, xdata, ydata , maxfev = 10000,
                              bounds=((-np.inf,0.0,-np.inf,0.0), (np.inf,np.inf,np.inf,np.inf)))
    elif method == "oneexp":
        popt, pcov = curve_fit(oneexp, xdata, ydata, maxfev = 10000, bounds = ((-np.inf, 0.0), (np.inf, np.inf)))
    return popt, pcov

    

In [ ]:
def find_Cl(r, xi,method):
    '''
    Compute Cl given discrete correlation funciton
    r: theta, in arcmin
    xi: must be kappa-kappa correlation
    method: method for the fitting function
    '''
    
    
    try:
        popt,_ = curve_fit_log(r/60.0, xi, method)
    except RuntimeError:
        print("fitting failed, change to xi=0")
        popt,_ = curve_fit_log(r/60.0, np.zeros(len(r)), method)

        
    # set up the theta list from 0-180 degree, and its cosine
    theta_list = np.arange(0.01,180,0.001)
    cosine_list = np.cos(theta_list/180.0*np.pi)

    # compute the best-fit xi in full range
    corr_list = []
    for theta in theta_list:
        if method =="powlaw":
            corr_list.append(powlaw(theta,*popt))
        elif method =="powtan":
            corr_list.append(powtan(theta,*popt))
        elif method =="twoexp":
            corr_list.append(twoexp(theta,*popt))
        elif method =="oneexp":
            corr_list.append(oneexp(theta,*popt))
    theta_rad = theta_list/180.0*np.pi
    corr_list = np.array(corr_list)
    weight = np.sin(np.array(theta_rad))*np.pi/corr_list.shape[0]

    corr_array = np.array([corr_list,corr_list,corr_list,corr_list]).T
    
    
    #Hankel transform
    Cl_camb = correlations.corr2cl(corr_array, cosine_list, weight, ell_max)
    
    
    
    #return ell_list_camb (1-3nside), Cl * ell * (ell+1)/ 2pi, best fit parameters
    return ell_list_camb[1:], Cl_camb[:,0][1:], popt

# Example if you already have Cl from 1 to 3NSIDE

## Simply call Cl2maps()


In [ ]:
nside = 1024

map_ = Cl2maps(Cl,nside)




In [ ]:
hp.mollzoom(map_, title = 'generated 1d map')

# Example if you already have xi from 1 to 200 arcmin

## 1. Visualize, decide which fitting function to use

## 2. find_Cl()

## 3. Cl2maps()


In [ ]:
#suppose our correlation function xi has theta = r

In [ ]:
nside = 1024

ell_array, Cl_camb, popt = find_Cl(r, xi, "oneexp")

map_ = Cl2maps(Cl_camb,nside, ell_factor = True)



In [ ]:
hp.mollzoom(map_, title = 'generated 1d map')

## Extra credits for measuring the correlation function of the generated maps, see if they matches your original input!

In [ ]:
##crop out a subset of the map, otherwise use too much memory

nside = 1024

(theta,phi) = hp.pixelfunc.pix2ang(nside,np.arange(0,hp.nside2npix(nside),1))
dec_map = theta*180.0/np.pi - 90.0
ra_map = phi*180.0/np.pi

index1 = dec_map>-20.0
index2 = dec_map<20.0
index3 = ra_map>-20.0
index4 = ra_map<20.0

index_total = index1 * index2 * index3 * index4
new_dec_map = dec_map[index_total!=0.0]
new_ra_map = ra_map[index_total!=0.0]
new_maps = maps[aindex_total!=0]


In [ ]:

# Store the cutout maps in an temprary directory

k_map_table = Table([new_ra_map,new_dec_map,new_maps, names=('ra', 'dec','kappa')])
write_table(k_map_table,'tmp/map_kappa.fits')
cat = treecorr.Catalog('tmp/map_kappa.fits', ra_col='ra', dec_col='dec', ra_units='deg', dec_units='deg', 
                       k_col='kappa', npatch = 10)

# Measure its auto correlation function

nbins = 20



kk_ij = treecorr.KKCorrelation(min_sep=1, max_sep=200, nbins=nbins, sep_units='arcmin')

kk_ij.process(cat, cat)

xi_maps = kk_ij.xi

xi_sig_maps = np.sqrt(kk_ij.varxi)

r_map = np.exp(kk_ij.meanlogr) 
        
        

In [ ]:
plt.figure(figsize = (8,6))

plt.plot(r,xi, fmt = '.', label = 'Input correlation function')
plt.plot(r_map, xi_maps,  label = 'Map correlation function')
plt.legend()

plt.xlabel('$\theta$ [arcmin]')
plt.ylabel('$\xi$')
plt.xscale('log')
plt.show()